In [6]:
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
import thinc.extra.datasets
import pandas as pd
import spacy
from spacy.util import minibatch, compounding
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from numpy import array, unique, array_equal
max_fatures = 1000


def load_data():
    print("Loading Data...")
    PrimaryEmotion = pd.read_csv('emotion.xls.csv')
    print("Data Loaded")
    return (PrimaryEmotion['sentence'], array(PrimaryEmotion['emotion']))



def sort_to_2_emotions(sentence_list, emotion_list):
    sorted_list = []
    sorted_emo = []
    for (data, emo) in zip(sentence_list, emotion_list):
        if (emo == 'Joy' or emo == 'Love' or emo == 'Optimism' or emo == 'Awe' or emo == 'Trust'):
            sorted_list.append(data)
            sorted_emo.append("positive")
        if (emo == 'anger' or emo == 'Disgust' or emo == 'Sadness' or emo == 'Aggression' or emo == 'Contempt' or emo == 'Disapproval' or emo == 'Remorse'):
            sorted_list.append(data)
            sorted_emo.append("negative")
    return (sorted_list, sorted_emo)



def tokenize(sentences):
    print("Tokenizing...")
    tokenizer = Tokenizer(num_words=max_fatures, split=' ')
    tokenizer.fit_on_texts(sentences)
    X = tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(X)
    print("Tokenization Completed")
    return X



def train(tokened_sentences, emotion_list):
    print("Training...")
    embed_dim = 128
    lstm_out = 196
    test_percent=.20

    model = Sequential()
    model.add(Embedding(max_fatures, embed_dim,input_length = tokened_sentences.shape[1]))
    model.add(SpatialDropout1D(0.4))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    
    # len(set(emotion_list)) is a hacky way of geting the number of unique elements
    # in a regualar python list (non-numpy)
    model.add(Dense(unique(emotion_list).size,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    #print(model.summary())

    Y = pd.get_dummies(emotion_list).values
    X_train, X_test, Y_train, Y_test = train_test_split(tokened_sentences,Y, test_size = test_percent, random_state = 42)
    #print(X_train.shape,Y_train.shape)
    #print(X_test.shape,Y_test.shape)

    batch_size = 32
    model.fit(X_train, Y_train, epochs = 100, batch_size=batch_size, verbose = 1)
    print("Training Completed")
    print("Testing Against Control... (% of the data) ", test_percent)
    score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
    print("Score   :", score)
    print("Accuracy:", acc)
    return (model, X_test)

### When Splitting data (train_test_split), we don't retain where in the 
### origional set the data is located, thus it takes
### a little trickery to see the results while while only testing against untrained data

def test(model, X_test, tokenized_data, sentence_list, emotion_list):
    predictions = model.predict(tokenized_data, batch_size=32)
    error = 0
    for i in range(len(predictions)):
     
        #  "If the current tokenized data array is in X_test (untrained tokenized arrays)
        
        if(any(array_equal(tokenized_data[i], x) for x in X_test)):
            pos = list(predictions[i]).index(max(predictions[i]))
            print("\n\n")
            print(sentence_list[i])
            print("\nPredicted emotion: ", unique(emotion_list)[pos])
            print("Actual emotion   : ", emotion_list[i])
            if (unique(emotion_list)[pos] != emotion_list[i]):
                error += 1
    print("%  Accuracy when against untrained set: ", 1- (float(error) / len(X_test)))

    
    
#################################
#############MAIN################
#################################
#   jupyter notebook is weird   #




(sentence_list, emotion_list) = load_data()


### Comment the Below line for all 18 emotions. This sorts into "positive" and "negative"
#sentence_list, emotion_list = sort_to_2_emotions(sentence_list, emotion_list)

tokenized_data = tokenize(sentence_list)

(model, X_test) = train(tokenized_data, emotion_list)

test(model, X_test, tokenized_data, sentence_list, emotion_list)


Loading Data...
Data Loaded
Tokenizing...
Tokenization Completed
Training...
Epoch 1/100
2019/2019 [==============================] - 11s 5ms/step - loss: 2.6401 - acc: 0.2060
Epoch 2/100
2019/2019 [==============================] - 9s 4ms/step - loss: 2.5667 - acc: 0.2070
Epoch 3/100
2019/2019 [==============================] - 9s 4ms/step - loss: 2.5144 - acc: 0.2209
Epoch 4/100
2019/2019 [==============================] - 9s 4ms/step - loss: 2.3895 - acc: 0.2660
Epoch 5/100
2019/2019 [==============================] - 9s 4ms/step - loss: 2.1781 - acc: 0.3512
Epoch 6/100
2019/2019 [==============================] - 9s 4ms/step - loss: 1.9636 - acc: 0.4106
Epoch 7/100
2019/2019 [==============================] - 9s 4ms/step - loss: 1.7883 - acc: 0.4705
Epoch 8/100
2019/2019 [==============================] - 9s 4ms/step - loss: 1.6370 - acc: 0.5136
Epoch 9/100
2019/2019 [==============================] - 9s 4ms/step - loss: 1.5016 - acc: 0.5458
Epoch 10/100
2019/2019 [================

2019/2019 [==============================] - 9s 4ms/step - loss: 0.1009 - acc: 0.9708
Epoch 83/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.1025 - acc: 0.9693
Epoch 84/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0846 - acc: 0.9723
Epoch 85/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0839 - acc: 0.9723
Epoch 86/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0825 - acc: 0.9708
Epoch 87/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0964 - acc: 0.9688
Epoch 88/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.1006 - acc: 0.9668
Epoch 89/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0933 - acc: 0.9703
Epoch 90/100
2019/2019 [==============================] - 9s 4ms/step - loss: 0.0930 - acc: 0.9713
Epoch 91/100
2019/2019 [==============================] - 11s 5ms/step - loss: 0.0974 - acc: 0.9663
Epoch 92/100
2019/2019




Eight extra bullets in a tight spot could mean the difference between life and death.

Predicted emotion:  Neutral
Actual emotion   :  Fear



But, somehow, he sensed that that would be an uphill battle.

Predicted emotion:  Anticipation
Actual emotion   :  Neutral



Yeah, Geoff. Tell him the coast is clear.

Predicted emotion:  Disapproval
Actual emotion   :  Neutral



Hoomey's teeth were chattering after his dice with death.

Predicted emotion:  Disapproval
Actual emotion   :  Fear



Obey them to the letter and you will be well rewarded -- any bungling or playing the fool and I doubt you'll live to tell the tale

Predicted emotion:  Fear
Actual emotion   :  Fear



It's a no go area.

Predicted emotion:  Fear
Actual emotion   :  Disapproval



I suppose it came into my room for a few minutes to see a book I'd been telling him about, then he left and I hit the sack.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



By the time he arrived at Amanda's he was ready to dro




I need to let off steam.

Predicted emotion:  Anger
Actual emotion   :  Anger



It's steamed up between the layers is that what you're looking at.

Predicted emotion:  Submission
Actual emotion   :  Submission



Dora accepted these ministrations without protest, but Melissa thought she detected a growing restiveness and a thinly veiled relief when at last they said goodnight and returned to their own room.

Predicted emotion:  Joy
Actual emotion   :  Submission



It's probably the aspirin sweating the chill out.

Predicted emotion:  Joy
Actual emotion   :  Neutral



I was absolutely in my element, how did I come over, Jannie?

Predicted emotion:  Remorse
Actual emotion   :  Fear



You're full of the joys of spring at the moment, so make the most of this harmonious period.

Predicted emotion:  Anticipation
Actual emotion   :  Joy



Pull the other one, Jo, it's got bells on it.

Predicted emotion:  Disapproval
Actual emotion   :  Anticipation



Mm, well it's supposed, they're d


Once that happens, according to the Meat and Livestock Commission, the floodgates will open and prices will go through the roof.

Predicted emotion:  Fear
Actual emotion   :  Anticipation



Most have been exterminated, and good riddance to them.

Predicted emotion:  Contempt
Actual emotion   :  Disgust



Usually she flashed back with a Mind Your Own Business, if anyone asked her a question.

Predicted emotion:  Neutral
Actual emotion   :  Contempt



Well, well, well, another humanoid to try my patience.

Predicted emotion:  Neutral
Actual emotion   :  Optimism



From July onwards the town of Whaddon had waited with bated breath.

Predicted emotion:  Ambiguous
Actual emotion   :  Anticipation



He longed for action, yearned to be trusted with high responsibilities, to undertake prodigious feats requiring a cool head, a stout heart and nerves of steel.

Predicted emotion:  Neutral
Actual emotion   :  Anticipation



Hold your horses, Mrs Parvis.

Predicted emotion:  Disapproval
Act

...




The fact we are not ahead of the pack at the moment is a help because everyone is not talking about us winning it.

Predicted emotion:  Contempt
Actual emotion   :  Optimism



But I love those little box things with the lacy paper that they pack it in.

Predicted emotion:  Neutral
Actual emotion   :  Love



I wasn't sure that our clothing would pass muster at the doors, but others in the crowd were even more outlandish and uncivilised.

Predicted emotion:  Joy
Actual emotion   :  Surprise



He had worked with Elizabeth in the 1951 picture A Place in the Sun and they had remained close friends.

Predicted emotion:  Optimism
Actual emotion   :  Love



The key behaviours that, even in the most difficult circumstance, make a win-win outcome.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



Everywhere in Greece the hoped-for growth of income from tourism depends in part on removing the smells that haunt too many beaches and other places where north Europeans twitch noses




Finally, under the obscurity of dust and smoke the Germans managed to get a foothold on the summit

Predicted emotion:  Neutral
Actual emotion   :  Joy



By mid-1945 the country was economically, militarily and psychologically on its knees, but unconditional surrender was still resisted.

Predicted emotion:  Submission
Actual emotion   :  Disapproval



And so the moment of truth.

Predicted emotion:  Optimism
Actual emotion   :  Surprise



Relying on a small bureau with perhaps a single film recorder is, as Murphy 's Law always points out, a somewhat foolish activity.

Predicted emotion:  Neutral
Actual emotion   :  Disapproval



A less than shattering experience simply won't pass muster in this of all Shostakovich symphonies.

Predicted emotion:  Anger
Actual emotion   :  Disapproval



Latecomers sink or swim in the great flotation race.

Predicted emotion:  Optimism
Actual emotion   :  Neutral



It is unforgivable to leave it to wither on the vine.

Predicted emotion:  Trust




The problem lies in sustaining the speech, for Mercutio is a hearty and boisterous character, seemingly at odds with the mischief and delicacy of this speech.

Predicted emotion:  Neutral
Actual emotion   :  Love



Microsoft Windows for Workgroups vs Lotus Notes looks set to be a long running battle.

Predicted emotion:  Contempt
Actual emotion   :  Neutral



Why couldn't God let bygones be bygones?

Predicted emotion:  Optimism
Actual emotion   :  Anger



However, once one begins to argue the toss about the liberal value of various academic subjects, it becomes difficult to exclude any of them.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Instead I was getting myself into an increasingly confrontational situation, a battle of wills, with the Director of Coaching.

Predicted emotion:  Neutral
Actual emotion   :  Aggression



Cut it out, Harvey.

Predicted emotion:  Disapproval
Actual emotion   :  Disapproval



If they can't accept me as I am, good riddance.

Pred




He had to learn through trial and error.

Predicted emotion:  Optimism
Actual emotion   :  Optimism



No I don't, but nevertheless, nobody knew only people like us and the neighbours, how that kid was every day when her mother according to Julia had her because it was the done thing.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Thought it has a supportive county council and a productive work force, yet it has its cross to bear.

Predicted emotion:  Neutral
Actual emotion   :  Remorse



But I won't leave you in the lurch even if that time comes.

Predicted emotion:  Anticipation
Actual emotion   :  Love



I didn't like that much because it was so smelly, but a necessary evil.

Predicted emotion:  Neutral
Actual emotion   :  Disgust



There is no quick fix to defeat terrorism in Northern Ireland.

Predicted emotion:  Neutral
Actual emotion   :  Neutral



Whatever you decide, it pays to look before you leap.

Predicted emotion:  Anticipation
Actual emotion   :  Neut




By this point some of my readers will be up in arms.

Predicted emotion:  Anger
Actual emotion   :  Neutral



Heavy dependence on sponsorship and arts council cash could spell trouble as the recession hits arts budgets.

Predicted emotion:  Fear
Actual emotion   :  Disapproval



In 1987, when I was chosen to fight my first election at Birmingham's Perry Barr, I was on cloud nine, even though it was a safe Labour seat.

Predicted emotion:  Anticipation
Actual emotion   :  Joy
%  Accuracy when against untrained set:  0.1841584158415842
